In [1]:
import requests
from config import req_headers, base_url, focus_league, local_host, local_name, local_port, local_user
from creds import local_db_password
import psycopg2
import time
import json

In [2]:
conn = psycopg2.connect(dbname=local_name, user =local_user, host=local_host, password=local_db_password, port =local_port)
cur = conn.cursor()

In [3]:
def get_max_id():
    max_id_sql = 'select max(job_id) from ffl_staging.query_data'
    cur.execute(max_id_sql)
    if cur.fetchall()[0][0] == None:
        return 0
    else:
        return cur.fetchall()[0][0] + 1

In [4]:
endpoint = 'fixtures/players'
url = base_url + endpoint
season = 2024

In [5]:
fixture_list = []
fetch_teams_sql = f"""select id from ffl.fixtures where tracking_status = 'posted' and date < now()"""
cur.execute(fetch_teams_sql)
for i in cur.fetchall():
    fixture_list.append(i[0])

In [6]:
fixture_insert_sql = """
insert into ffl_staging.query_data (
    job_id,
    query_scope,
    response_data,
    status
)
values
(%s,%s,%s, %s)
on conflict do nothing
"""

In [7]:
def parser(data=None, id_counter = None):
    job_id = id_counter
    query_scope = endpoint
    response_data = data


    return job_id, query_scope, json.dumps(response_data), 'loaded'

In [8]:
def fetch_response(params=None):
    req = requests.get(url, params=params, headers=req_headers)

    return req.json()

def load_responses(response_data=None, count_pass=None):
    cur.execute(fixture_insert_sql, (parser(response_data, count_pass)))

In [9]:
update_sql = """
update ffl.fixtures
set tracking_status = 'player_queried'
where id = %s
"""

In [10]:
counter = get_max_id()
for fixture in fixture_list:
    call_params = {
        'fixture':fixture
    }
    
    load_responses(fetch_response(call_params), counter)
    conn.commit()
    counter += 1
    cur.execute(update_sql, (fixture,))
    conn.commit()